In [11]:
from collections.abc import Sequence
from __future__ import annotations
from numbers import Number



In [12]:
from itertools import combinations
from typing import Callable
from typing import Type
from typing import Tuple
#from collections.abc import Iterable 
#@abstractmethod
#    @property
#    @abstractmethod
# no need to call super
from icecream import ic
    
    
from abc import ABC, abstractmethod
import numpy as np
        
class function(ABC):
    @property
    @abstractmethod
    def forward_func(self) -> Callable:
        pass
    
    @property
    @abstractmethod
    def backward_func(self) -> Callable:
        pass
    
    def get_functions(self) -> Tuple[Callable, Callable]:
        return self.forward_func, self.backward_func
    
    def __call__(self, *inner) -> expr_node: # simplifies the syntax for chaning of functions
        return expr_node(self, inner)
        
    

class sin(function):
    forward_func = np.sin
    backward_func = np.cos

class tanh(function):
    forward_func = np.tanh
    backward_func = lambda x: 1 - np.square(tanh(x))

class cos(function):
    forward_func = np.cos
    backward_func = lambda x: -np.sin(x)
          

class multiply(function):
    
    forward_func = lambda: None # to be overwitten in constructor
    backward_func = lambda: None # to be overwitten in constructor
    
    def __init__(self,*, allow_arbitrary_many = False): # * makes allow_arbitary_many to keword only
        if not allow_arbitrary_many: # simple case two factors
            self.forward_func = np.multiply
            self.backward_func = lambda x, y: (y, x)
        else:
            forward_func = lambda *x: np.prod(np.vstack(x), axis=0)
            self.backward_func = lambda *x: (np.prod(np.vstack(values), axis=0) for values in combinations(x, len(x) - 1))
                   
            
        
class add(function):
    
    forward_func = lambda: None # to be overwitten in constructor
    backward_func = lambda: None # to be overwitten in constructor
    
    def __init__(self,*, allow_arbitrary_many = False): # new is clalled before the constructer (before: "__init__")
        if not allow_arbitrary_many:
            self.forward_func = np.add 
            self.backward_func = lambda x, y: (np.full_like(x, 1), np.full_like(y, 1)) #arrays with same shape of x and y, filled with 1
        else:
            self.forward_func = lambda *x: np.sum(np.vstack(x), axis=0)
            self.backward_func = lambda *x: (np.full_like(x[0], 1) for _ in range(len(x))) #backward_func = lambda *x: (np.ones(len(x[0]) if isinstance(x[0], Iterable) else 1) for _ in range(len(x))))


class tanh(function):

    forward_func = np.tanh
    backward_func = lambda x: 1/(cosh(x)**2)
              
    
#b = function2(np.sin, np.cos)
c = sin()
print(c, c.forward_func, c.backward_func)
f1, f2 = c.get_functions()
print(f1, f2)
#c.eval_forward(2)
d = [sin(), sin()]


e = multiply()
print(e.forward_func)


e = multiply(allow_arbitrary_many=True)
print(e.forward_func)
e = multiply(allow_arbitrary_many=False)
print(e.forward_func)
f = cos()
print(f.forward_func(2), np.cos(2))  
forw, backw = c.get_functions()
print(forw, backw) 
ic.disable()
ghj = ic(f.forward_func(2))
ic(ghj)
k = tanh()
l = cos()
ic.enable()
ic(l.forward_func(2))
ic()

        

ic| l.forward_func(2): -0.4161468365471424
ic| 2492483270.py:111 in <module> at 11:35:55.519


<__main__.sin object at 0x7f1754eeb620> <ufunc 'sin'> <ufunc 'cos'>
<ufunc 'sin'> <ufunc 'cos'>
<ufunc 'multiply'>
<bound method multiply.<lambda> of <__main__.multiply object at 0x7f1754c69950>>
<ufunc 'multiply'>
-0.4161468365471424 -0.4161468365471424
<ufunc 'sin'> <ufunc 'cos'>


In [13]:
class expr_end_node():
    def __init__(self, value: Number | np.ndarray , grad_value: Number | np.ndarray  = 0):
        self.value = value
        self.grad_value = grad_value
        
class expr_node():
    def __init__(self, func: function, childs: Sequence[expr_node | expr_end_node] = []):
        #self.parants = parants #expr_node
        self.childs = childs  #expr_node
        self.func = func #function 
        # lieber so:
        # self.forward_func, self.backward_func = func.get_functions()
        # und dann self.func entfernen
        
f = sin()
g = sin()
ex1 = expr_node(f)
ex2 = expr_node(g)
ex3 = expr_node(g)
ex1.childs = [ex2]
#ex2.parants = [ex1]
ex2.childs = [ex3]
#sin(sin(sin()))

In [14]:
ex1 = expr_node(sin())
ex2 = expr_node(sin())
ex3 = expr_node(sin())
x = expr_end_node(np.float64(0.345))
ex1.childs = [ex2]
ex2.childs = [ex3]
ex3.childs = [x]

#sin(sin(sin(x)))
"""
sin
 |
sin
 |
sin
 |
 x=0.345
"""


#cos(a)*sin(b + tanh(c))
example_expr2 = expr_node(multiply(), 
                          [expr_node(cos(),
                                     [expr_end_node(np.float64(0.345))]
                                    ), 
                           expr_node(sin(),
                                     [expr_node(add(), 
                                                [expr_end_node(np.float64(0.345)),
                                                 expr_node(tanh(), 
                                                           [expr_end_node(np.float64(0.345))]
                                                          )
                                                ]
                                               )
                                     ]
                                    )
                          ]
                         )

# the same using the __call__ method of function (see above in function definition):
example_expr3 = multiply()(
                        cos()(
                            expr_end_node(np.float64(0.345))),
                            sin()(
                                add()(
                                    expr_end_node(np.float64(0.345)),
                                    tanh()(
                                        expr_end_node(np.float64(0.345))
                                    )
                                )
                            )
                )
                
"""
    multiply
    |      |
   cos    sin
   |       |
  a=0.345 add
          | |
    b=0.345 tanh
             |
         c=0.345
"""
#short version of the sin(sin(sin(x))) exaple:
x = expr_end_node(np.float64(0.345))
example_expr1 = sin()( sin()( sin()( x ) ) ) 






In [15]:
#!source .jupyter_venv/.venv/bin/activate # source venv, if you have one for your kernal
#import sys
#!{sys.executable} -m pip install graphviz

import graphviz


def print_graph(node, graph, parent_id=""):
    
    node_id = str(id(node))
    node_label = str(node.value) if isinstance(node, expr_end_node) else type(node.func).__name__
    graph.node(node_id, node_label)  
    
    if parent_id:
        graph.edge(parent_id, node_id)
        
    if hasattr(node, 'childs') and node.childs:
            for child in node.childs:
                print_graph(child, graph, node_id)#, constraint='false')


dot1 = graphviz.Digraph('the_graph', comment='test') 
dot1.attr(rankdir="LR")
print_graph(example_expr3, dot1)
#doctest_mark_exe() 
dot1.render(directory='graph_out/tt', view=True)
#dot  
#str(id(ex1))
#type(ex1.func).__name__

'graph_out/tt/the_graph.gv.pdf'

In [16]:

#f(g(h(x)))' = f'(g(h(x))) * (g(h(x)))' =  f'(g(h(x))) * g'(h(x)) * h'(x)
# h'(x) * g'(h(x)) * f'(g(h(x)))


def forward(node):
    return node.func.forward_func(*(forward(child) for child in node.childs)) if type(node) is not expr_end_node else node.value 
forward(ex1)
"""
def forward(node):
    if type(node) is not expr_end_node:
    
        node.func.forward_func(*(forward(child) for child in node.childs))
    else:
        node.value 
"""
forward(ex1)

def backward(node, value = 1):
    if type(node) is not expr_end_node:
        child_values = (forward(child) for child in node.childs)
        if len(node.childs) == 1:
            backward(node.childs[0], value * node.func.backward_func(*child_values))
        else:
            for child, new_value in zip(node.childs, node.func.backward_func(*child_values), strict=True):
                backward(child, value * new_value)
    else: 
        node.grad_value += value
        
        
#result,target
    
        
        
x.grad_value=0

ic(forward(ex1))
ic(np.sin(np.sin(np.sin(np.float64(.345)))))

backward(ex1)
ic(x.grad_value)
ic(np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345))

print(x.grad_value)


"""
def print_all(a,b,c):
    print(a)
    print(b)
    print(c)

print_all(*[1,2,3])
"""
#zip((a1,a2.a3),(b1,b2,b3) -> ((a1,b1),(a2,b2),(a3,b3))


ic| forward(ex1): 0.32573257710464837
ic| np.sin(np.sin(np.sin(np.float64(.345)))): 0.32573257710464837
ic| x.grad_value: 0
ic| np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345): 0.8393506506551831


0


'\ndef print_all(a,b,c):\n    print(a)\n    print(b)\n    print(c)\n\nprint_all(*[1,2,3])\n'

In [17]:
from sklearn.datasets import load_iris
from numpy import linalg as LA



iris = load_iris()
print(len(iris.data), len(iris.target))
print(type(iris.data), type(iris.target))
ic(iris.target)
normalized_data = iris.data/LA.norm(iris.data, axis=0)

"""
out = vector len 3 , out_1: suit 1,  out_1: suit 2, 
suit_1 = (1,0,0)
suit_2 = (0,1,0)
suit_3 = (0,0,1)

netz = 
    in: vecor len 4 
    out = vector len 3
    
with  2-5 hidden layer; 4- 10 neurons each
"""

ic| iris.target: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

150 150
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


'\nout = vector len 3 , out_1: suit 1,  out_1: suit 2, \nsuit_1 = (1,0,0)\nsuit_2 = (0,1,0)\nsuit_3 = (0,0,1)\n\nnetz = \n    in: vecor len 4 \n    out = vector len 3\n    \nwith  2-5 hidden layer; 4- 10 neurons each\n'

In [18]:
test_prediction = np.array([0.2, 0.6, 0.3])
test_target = np.array([0, 1, 0])

# Mean squared error (MSE):
def loss_mse(prediction, target):
    return np.square(prediction - target).sum()
#(pred_x, pred_y), (target_x, target_y) -> (pred_x - target_x)²+(pred_y - target_y)²



loss1 = ic(loss_mse(test_prediction, test_target))
ic(np.log(test_prediction))
# loss2 = ic(loss_ce(test_prediction,test_target))

#ic.disable()
#ic.enable()

# 0.2²+0.4²+0.3² = 
#ic(loss1(test_prediction, test_target))


ic| loss_mse(test_prediction, test_target): 0.29000000000000004
ic| np.log(test_prediction): array([-1.60943791, -0.51082562, -1.2039728 ])


array([-1.60943791, -0.51082562, -1.2039728 ])

In [5]:
# in_1     out_1
# in_2     out_2

# initial = w, b  are random values
# netz(in) = 
#     out_1 = in_1 * w_{1,1} + in_2 *w_{1,2} + b_1
#     out_2 = in_1 * w_{2,1} + in_2 *w_{2,2} + b_2

# # net(in, {w}, {b}) 
# out = netz(in)
# loss(out, true_val)
# backpropergation -> change  w, b <--------



# dict(np.sin: np.cos)

'\ninitial = w, b  are random values\nnetz(in) = \n    out_1 = in_1 * w_{1,1} + in_2 *w_{1,2} + b_1\n    out_2 = in_1 * w_{2,1} + in_2 *w_{2,2} + b_2\n\n# net(in, {w}, {b}) \nout = netz(in)\nloss(out, true_val)\nbackpropergation -> change  w, b <--------\n\n\n\ndict(np.sin: np.cos)\n'

In [1]:
# from http://neuralnetworksanddeeplearning.com/chap1.html
# only as reference, should eventually be adjusted for our code 
# sigmoid changed to tanh

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = tanhyper(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            tanhyperprime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)


def tanhyper(x):
    return np.tanh(x)

def tanhyperprime(x):
    return 1/cosh(x)**2
    